In [1]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os
import TrainDataPrepare as td
import HelperFunctions as hf

sys.path.remove('../')

In [2]:
#prepare csv
#prepare data for traning and place them into prepared_data folder
td.SelectDataByPercent('../OOC_image_dataset/train', '../prepared_data/train', 100)
td.SelectDataByPercent('../synthetic_data_0.1/train', '../prepared_data/train', 25)

#transform xlsx file to csv 
hf.convert_xlsx_to_csv('../OOC_datasheet.xlsx', '../OOC_datasheet.csv')

#create csv file also for sythetic data 
td.create_synthetic_csv_from_real_and_structure('../OOC_datasheet.csv', '../synthetic_data_0.1/train', '../OOC_datasheet_01.csv')

#join both csv files toghether
hf.concatenate_csv('../OOC_datasheet.csv', '../OOC_datasheet_01.csv', '../OOC_datasheet_joined.csv')
hf.delete_file('../OOC_datasheet.csv')
hf.delete_file('../OOC_datasheet_01.csv')

#transfer val and test datasets to prepared data
hf.copy_directory('../OOC_image_dataset/test/', '../prepared_data')
hf.copy_directory('../OOC_image_dataset/val/', '../prepared_data')

#split the csv files into train, val, test
hf.split_datasets_by_actual_images('../OOC_datasheet_joined.csv', '../prepared_data', '../prepared_data')
hf.delete_file('../OOC_datasheet_joined.csv')



Headers in the CSV: ['imageID', 'cell type', 'seeding density, cells/ml', 'time after seeding, h', 'day', 'Decision 1/2 (good/bad)', 'flow rate']
Copied 'c:\Users\Roberts\Documents\repos\Bakalaurs\OOC_image_dataset\test' to 'c:\Users\Roberts\Documents\repos\Bakalaurs\prepared_data\test' successfully.
Copied 'c:\Users\Roberts\Documents\repos\Bakalaurs\OOC_image_dataset\val' to 'c:\Users\Roberts\Documents\repos\Bakalaurs\prepared_data\val' successfully.
Collected 4684 image paths.
Filtered dataset contains 4684 records.
Train data saved to ../prepared_data\train\train_dataset.csv
Val data saved to ../prepared_data\val\val_dataset.csv
Test data saved to ../prepared_data\test\test_dataset.csv


In [6]:



transform = transforms.Compose([
    transforms.Resize((600, 600)),  # EfficientNet B7 uses a larger input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])





# Create datasets
train_dataset = td.CellsDataset(root_dir='../OOC_image_dataset/train', transform=transform)
val_dataset = td.CellsDataset(root_dir='../OOC_image_dataset/val', transform=transform)
test_dataset = td.CellsDataset(root_dir='../OOC_image_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


